# Introducción

En este proyecto, se aborda el problema de la evaluación de similitud semántica entre pares de oraciones. Este es un desafío fundamental en el procesamiento del lenguaje natural (NLP), con aplicaciones en tareas como recuperación de información, detección de plagio, sistemas de recomendación y más.

Este trabajo forma parte de la materia **Deep Learning para NLP**, donde se exploran técnicas avanzadas para resolver problemas complejos en el ámbito del lenguaje natural.

Para resolver este problema, se implementan y comparan tres enfoques basados en modelos de lenguaje preentrenados (BERT y variantes optimizadas para similitud semántica). Los modelos utilizados son:

1. **BERT + Regresión**: Un modelo basado en BERT con una capa de regresión para predecir la similitud.
2. **Siamese BERT**: Un modelo siamés que compara las representaciones de dos oraciones.
3. **Cross-Attention Model**: Un modelo que utiliza atención cruzada para capturar interacciones entre las oraciones.

El objetivo es entrenar y evaluar estos modelos en el conjunto de datos STS Benchmark, normalizando las etiquetas a un rango de 0 a 1. Finalmente, se validan los modelos con ejemplos de prueba para analizar su desempeño en diferentes niveles de similitud semántica.

In [ ]:
import torch
import numpy as np
import random
import evaluate
import pandas as pd
import textwrap
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from datasets import load_dataset
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, get_scheduler
from transformers import DataCollatorWithPadding

import ipywidgets as widgets
from IPython.display import display

Configuracion para la Reproducibilidad

In [ ]:
# Seed para asegurar reproducibilidad sin importar el dispositivo en el que se ejecute
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

### Carga y Preprocesamiento del Dataset

In [ ]:
# Cargar el dataset
dataset = load_dataset("mteb/stsbenchmark-sts")

# Normalizar las etiquetas (de 0-5 a un rango de 0-1)
def normalize_labels(examples):
    examples["score"] = [s / 5.0 for s in examples["score"]]
    return examples

# Aplicar normalización
dataset = dataset.map(normalize_labels, batched=True)

Tokenizacion del Texto

In [ ]:
#Creamos un tokenizador para convertir las frases en IDs de tokens que el modelo pueda procesar. Usare BERT o un modelo optimizado para similitud semántica
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")

#Definiendo la funcion de tokenizacion
def tokenize_function(examples):
    #Tokenizando sentence1 y sentence2 por separado
    #Esto es necesario porque el modelo espera que las entradas sean pares de oraciones
    #y no una sola oración.
    encodings1 = tokenizer(
        examples['sentence1'],
        padding =False,
        truncation=True,
        max_length=128,
        return_tensors=None # Lo dejo como lista para el dataset
    )
    encodings2 = tokenizer(
        examples['sentence2'],
        padding="False",
        truncation=True,
        max_length=128
        return_tensors=None # Lo dejo como lista para el dataset
    )
    return{
        'input_ids1': encodings1['input_ids'],
        'attention_mask1': encodings1['attention_mask'],
        'input_ids2': encodings2['input_ids'],
        'attention_mask2': encodings2['attention_mask'],
        'score': examples['score'] # Mantengo la etiqueta original
    }
    
#Tokenizando el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Probando 123...

In [ ]:
print(tokenized_datasets["train"][0]["input_ids1"])
print(tokenized_datasets["train"][0]["input_ids2"])
print(len(tokenized_datasets["train"][0]["input_ids1"]), len(tokenized_datasets["train"][0]["input_ids2"]))

Creando el Dataset para Pytorch

In [ ]:
#Creo una clase para formatear los datos de entrada correctamente para PyTorch
class STSDataset(Dataset):
    def __init__(self, data):        
        self.input_ids = data["input_ids"]
        self.attention_mask = data["attention_mask"]
        self.scores = data["score"]  # Los scores son las etiquetas (labels)

        
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return{
            'input_ids1': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask1': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'input_ids2': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask2': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.scores[idx], dtype=torch.float),
        }
#Creando el datase de entrenamiento y validacion
train_dataset = STSDataset(tokenized_datasets["train"])
val_dataset = STSDataset(tokenized_datasets["validation"])

# Creando la Funcion para Fine-Tuning en los Modelos

In [ ]:
# Se crea un Widget para diferentes parametros de configuracion e identificar la mejor opcion.
learning_rate = widgets.FloatText(value=1e-5, description="learning_rate:")
freeze_layers = widgets.IntText(value=4, description="freeze_layers:")
epochs = widgets.IntText(value=20, description="epochs:")
scheduler_type = widgets.Text(value='linear', description="scheduler_type:")

display(learning_rate, freeze_layers, epochs, scheduler_type)


lr = learning_rate.value
fl = freeze_layers.value
ep = epochs.value
st = scheduler_type.value


FloatText(value=1e-05, description='learning_rate:')

IntText(value=4, description='freeze_layers:')

IntText(value=20, description='epochs:')

Text(value='linear', description='scheduler_type:')

In [ ]:
# Función para congelar capas
def freeze_bert_layers(model, num_frozen_layers=fl):
    """
    Congela las primeras `num_frozen_layers` capas del modelo BERT.
    """
    for layer in model.bert.encoder.layer[:num_frozen_layers]:
        for param in layer.parameters():
            param.requires_grad = False

    # Congelamos las embeddings iniciales también
    for param in model.bert.embeddings.parameters():
        param.requires_grad = False


### Definiendo los Modelos

Modelo 1 : Siamese BERT

In [ ]:
#Definiendo el primer modelo a utilizar, en este caso estaremos utilizando el modelo de Sentence Transformers "all-MiniLM-L6-v2"
#Creo un modelo basado en BERT con un head de regresión
class SentenceSimilarityModelOne(torch.nn.Module):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", freeze_layers=fl):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        
        # Congelar capas según parámetro freeze_layers
        for i, param in enumerate(self.bert.encoder.layer):
            if i < (len(self.bert.encoder.layer) - freeze_layers):
                for p in param.parameters():
                    p.requires_grad = False
                    
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(384, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1)
        )
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        similarity = self.regressor(cls_embeddings)
        return similarity.squeeze()

Modelo 2: Siamese BERT

In [ ]:
class SiameseBERT(torch.nn.Module):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", freeze_layers=fl):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        
        # Congelar capas según parámetro freeze_layers
        for i, param in enumerate(self.bert.encoder.layer):
            if i < (len(self.bert.encoder.layer) - freeze_layers):
                for p in param.parameters():
                    p.requires_grad = False
           
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(384 * 3, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1)
        )

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        output1 = self.bert(input_ids1, attention_mask=attention_mask1).last_hidden_state[:, 0, :]
        output2 = self.bert(input_ids2, attention_mask=attention_mask2).last_hidden_state[:, 0, :]

        diff = torch.abs(output1 - output2)
        mult = output1 * output2
        combined = torch.cat([diff, mult, output1], dim=1)

        return self.regressor(combined).squeeze()


Modelo 3: Cross-Attention Model

In [ ]:
class CrossAttentionModel(torch.nn.Module):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", freeze_layers=fl):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        
        # Congelar capas según parámetro freeze_layers
        for i, param in enumerate(self.bert.encoder.layer):
            if i < (len(self.bert.encoder.layer) - freeze_layers):
                for p in param.parameters():
                    p.requires_grad = False
           
        self.attention = torch.nn.MultiheadAttention(embed_dim=384, num_heads=8)
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(384 * 2, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1)
        )

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        output1 = self.bert(input_ids1, attention_mask=attention_mask1).last_hidden_state
        output2 = self.bert(input_ids2, attention_mask=attention_mask2).last_hidden_state

        attn_output, _ = self.attention(output1, output2, output2)
        combined = torch.cat([output1[:, 0, :], attn_output[:, 0, :]], dim=1)

        return self.regressor(combined).squeeze()


### Configuracion el Entrenamiento

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = SentenceSimilarityModelOne().to(device)
model2 = SiameseBERT().to(device)
model3 = CrossAttentionModel().to(device)

for model in [model1, model2, model3]:
    optimizer = AdamW(model.parameters(), lr=lr)
    loss_fn = torch.nn.MSELoss()


Validando el Fine - Tuning de los Modelos

In [ ]:
def print_trainable_params(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total Parámetros: {total_params:,}, Entrenables: {trainable_params:,}")

print("\nBERT + Regresión:")
print_trainable_params(model1)

print("\nSiamese BERT:")
print_trainable_params(model2)

print("\nCross-Attention:")
print_trainable_params(model3)



BERT + Regresión:
Total Parámetros: 22,812,033, Entrenables: 19,263,105

Siamese BERT:
Total Parámetros: 23,008,641, Entrenables: 19,459,713

Cross-Attention:
Total Parámetros: 23,501,697, Entrenables: 19,952,769


### Evaluación del Modelo

In [ ]:
# Función de evaluación
def evaluate_model(model, val_loader, model_type="bert"):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            if model_type == "bert":
                inputs = batch["input_ids1"].to(device)
                mask = batch["attention_mask1"].to(device)
                lbls = batch["labels"].to(device)
                outputs = model(inputs, mask)

            elif model_type in ["siamese", "cross"]:
                inputs1 = batch["input_ids1"].to(device)
                mask1 = batch["attention_mask1"].to(device)
                inputs2 = batch["input_ids2"].to(device)
                mask2 = batch["attention_mask2"].to(device)
                lbls = batch["labels"].to(device)

                outputs = model(inputs1, mask1, inputs2, mask2)

            preds.extend(outputs.cpu().numpy())
            labels.extend(lbls.cpu().numpy())

    correlation, _ = pearsonr(preds, labels)
    print(f"{model_type.upper()} - Pearson Correlation: {correlation:.4f}")
    return correlation

In [ ]:
def weighted_mse_loss(output, target):
    weights = torch.where(target < 1,2.0,1.0) # para similitudes muy bajas, se penaliza más
    return torch.mean(weights * (output - target) ** 2)

# Función para entrenar modelo con diferentes configuraciones
def train_model(model, train_loader, val_loader, model_type="bert", epochs=ep):
    model.train()
    optimizer = AdamW(model.parameters(), lr=lr) #
    loss_fn = torch.nn.MSELoss()
    scheduler = get_scheduler(st, optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs)
    history = []

    for epoch in range(epochs):
        total_loss = 0
        model.train()
        
        for batch in train_loader:
            optimizer.zero_grad()

            if model_type == "bert":
                inputs = batch["input_ids1"].to(device)
                mask = batch["attention_mask1"].to(device)
                labels = batch["labels"].to(device)
                outputs = model(inputs, mask)

            elif model_type in ["siamese", "cross"]:
                inputs1 = batch["input_ids1"].to(device)
                mask1 = batch["attention_mask1"].to(device)
                inputs2 = batch["input_ids2"].to(device)
                mask2 = batch["attention_mask2"].to(device)
                labels = batch["labels"].to(device)
                outputs = model(inputs1, mask1, inputs2, mask2)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
         # Evaluo después de cada época
        pearson_corr = evaluate_model(model, val_loader, model_type)
        history.append((avg_train_loss, pearson_corr))
        print(f"{model_type.upper()} - Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.4f}")

    print(f"{model_type.upper()} - Entrenamiento finalizado.")
    
    return history # Devuelvo el historial de pérdidas y correlaciones

In [ ]:
def infer_similarity(model, tokenizer, sentence1, sentence2):
    model.eval()
    with torch.no_grad():
        encoded_input = tokenizer(
            sentence1,
            sentence2,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)

        if isinstance(model, SiameseBERT) or isinstance(model, CrossAttentionModel):
            input_ids1 = encoded_input["input_ids"]
            attention_mask1 = encoded_input["attention_mask"]
            input_ids2 = encoded_input["input_ids"]
            attention_mask2 = encoded_input["attention_mask"]

            similarity = model(input_ids1, attention_mask1, input_ids2, attention_mask2)
        else:
            similarity = model(encoded_input["input_ids"], encoded_input["attention_mask"])

    return similarity.item() * 5  # Volvemos a la escala de 0 a 5


Validando los 6 ejemplo contra los modelos entrenados

In [ ]:
test_sentences = [
    ("I love eating apples", "The capital of France is Paris"),  # Similitud 0 (Completamente distintas)
    ("I have a black cat", "My pet is a dog"),  # Similitud 1 (Diferentes pero relacionadas con mascotas)
    ("He plays soccer on weekends", "She enjoys playing tennis on Sundays"),  # Similitud 2 (Acciones similares, pero no iguales)
    ("The sun is shining in the sky", "It is a bright and sunny day"),  # Similitud 3 (Mismo contexto, expresado diferente)
    ("The smartphone has a large screen", "This phone features a big display"),  # Similitud 4 (Misma idea, palabras diferentes)
    ("The dog is barking loudly", "The dog is barking loudly"),  # Similitud 5 (Frases idénticas)
]

models = {
    "BERT + Regression": model1,
    "Siamese BERT": model2,
    "Cross-Attention": model3
}

for model_name, model in models.items():
    print(f"\nEvaluando {model_name}:")
    for i, (s1, s2) in enumerate(test_sentences):
        score = infer_similarity(model, tokenizer, s1, s2)
        print(f"Ejemplo {i+1}: {s1} | {s2} → Score: {score:.2f} --> {round(score)}")



Evaluando BERT + Regression:
Ejemplo 1: I love eating apples | The capital of France is Paris → Score: 0.50 --> 1
Ejemplo 2: I have a black cat | My pet is a dog → Score: 0.74 --> 1
Ejemplo 3: He plays soccer on weekends | She enjoys playing tennis on Sundays → Score: 0.46 --> 0
Ejemplo 4: The sun is shining in the sky | It is a bright and sunny day → Score: 4.03 --> 4
Ejemplo 5: The smartphone has a large screen | This phone features a big display → Score: 3.91 --> 4
Ejemplo 6: The dog is barking loudly | The dog is barking loudly → Score: 5.05 --> 5

Evaluando Siamese BERT:
Ejemplo 1: I love eating apples | The capital of France is Paris → Score: 0.68 --> 1
Ejemplo 2: I have a black cat | My pet is a dog → Score: 0.97 --> 1
Ejemplo 3: He plays soccer on weekends | She enjoys playing tennis on Sundays → Score: 1.19 --> 1
Ejemplo 4: The sun is shining in the sky | It is a bright and sunny day → Score: 4.11 --> 4
Ejemplo 5: The smartphone has a large screen | This phone features a big 